In [ ]:
# to download the model checkpoints if they are not already present

# use this https://github.com/shivammehta25/Matcha-TTS-checkpoints/releases/download/v1.0/matcha_ljspeech.ckpt   
# https://github.com/shivammehta25/Matcha-TTS-checkpoints/releases/download/v1.0/generator_v1

In [1]:
import os
os.environ['PHONEMIZER_ESPEAK_LIBRARY'] = r'C:\Program Files\eSpeak NG\libespeak-ng.dll' # path to espeak-ng dll on Windows

In [6]:
import sys
import os
import torch
from process_text import process_text 

sys.path.insert(0, os.path.join(os.getcwd(), 'Matcha_TTS_main')) # Matcha_TTS_main is the code from the original repo 
from matcha.models.matcha_tts import MatchaTTS


# for Hifigan
from matcha.hifigan.config import v1
from matcha.hifigan.denoiser import Denoiser
from matcha.hifigan.env import AttrDict
from matcha.hifigan.models import Generator as HiFiGAN

In [7]:
# load the model checkpoints
matcha_checkpoint_path = "checkpoints/matcha_ljspeech.ckpt"
hifigan_checkpoint_path = "checkpoints/generator_v1"

In [8]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# load the matcha tts model

def load_model(checkpoint_path):
    model = MatchaTTS.load_from_checkpoint(
        checkpoint_path, 
        map_location=device,
        weights_only=False 
    )
    model.eval()
    return model

count_params = lambda x: f"{sum(p.numel() for p in x.parameters()):,}"

model = load_model(matcha_checkpoint_path)
print(f"Model loaded! Parameter count: {count_params(model)}")

c:\Users\arezk\Desktop\M2\this year\ML\Projet\matcha-tts-code\matcha-tts\matcha_env\lib\site-packages\diffusers\models\lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)


Model loaded! Parameter count: 18,204,193


In [10]:
# load the hifigan vocoder model

def load_vocoder(checkpoint_path):
    h = AttrDict(v1)
    hifigan = HiFiGAN(h).to(device)
    hifigan.load_state_dict(torch.load(checkpoint_path, map_location=device)['generator'])
    _ = hifigan.eval()
    hifigan.remove_weight_norm()
    return hifigan

vocoder = load_vocoder(hifigan_checkpoint_path)
denoiser = Denoiser(vocoder, mode='zeros')

c:\Users\arezk\Desktop\M2\this year\ML\Projet\matcha-tts-code\matcha-tts\matcha_env\lib\site-packages\torch\nn\utils\weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Removing weight norm...


## the pipline is this : 
Text → process_text() → synthesise() → mel-spectrogram
                                      ↓
                              to_waveform() → audio waveform
                                      ↓
                              save_to_folder() → .wav file

In [12]:
# process_text

test_text = "Hello, Mr. Smith! How are you?"
result = process_text(test_text, device)

print (result)

Processing complete!
{'x_orig': 'Hello, Mr. Smith! How are you?', 'x_phones': 'həlˈoʊ, mˈɪstɚ smˈɪθ! hˈaʊ ɑːɹ juː?', 'x': tensor([[ 50,   0,  83,   0,  54,   0, 156,   0,  57,   0, 135,   0,   3,   0,
          16,   0,  55,   0, 156,   0, 102,   0,  61,   0,  62,   0,  85,   0,
          16,   0,  61,   0,  55,   0, 156,   0, 102,   0, 119,   0,   5,   0,
          16,   0,  50,   0, 156,   0,  43,   0, 135,   0,  16,   0,  69,   0,
         158,   0, 123,   0,  16,   0,  52,   0,  63,   0, 158,   0,   6]]), 'x_lengths': tensor([69]), 'sequence': [50, 83, 54, 156, 57, 135, 3, 16, 55, 156, 102, 61, 62, 85, 16, 61, 55, 156, 102, 119, 5, 16, 50, 156, 43, 135, 16, 69, 158, 123, 16, 52, 63, 158, 6]}
